# Getting started with tket2

This demo notebook gives an overview of currently implemented tket2 features.

Be aware that the library is still in development and some features may not be
fully implemented or may change in the future.



## Installation

To install the library, you can use pip:

In [1]:
%pip install tket2

Note: you may need to restart the kernel to use updated packages.


In [9]:
from tket2.circuit import Tk2Circuit
import math

Let's configure pretty printing for the circuits, using the mermaid renderer.
This will render the circuit graphs in `jupyter-lab`, but it is not currently supported when viewing the library in vscode.

In [10]:
from tket2.circuit import render_circuit_mermaid

setattr(
    Tk2Circuit,
    "_repr_markdown_",
    lambda self: f"```mermaid\n{render_circuit_mermaid(self)}\n```"
)

# Defining circuits

There are multiple ways for defining circuits in tket2.
The library provides two limited builders, and it supports importing circuits from `guppy` and `pytket`.

### Using the commands-based builder

The simplest way is to use the commands-based builder interface `CircBuild`.
It supports constructing circuits by listing a series of commands applied to specific qubits.

In [11]:
from tket2.circuit.build import CircBuild, H, CX

builder = CircBuild.with_nqb(2)

builder.extend(H(0), CX(0, 1))
builder.set_tracked_outputs()
circ = builder.finish()
circ

```mermaid
graph LR
    subgraph 0 ["(0) DFG"]
        direction LR
        1["(1) Input"]
        2["(2) Output"]
        3["(3) quantum.tket2.H"]
        4["(4) quantum.tket2.CX"]
        1--"0:0<br>qubit"-->3
        1--"1:1<br>qubit"-->4
        3--"0:0<br>qubit"-->4
        4--"0:0<br>qubit"-->2
        4--"1:1<br>qubit"-->2
    end

```

### Using the Dataflow Builder

`CircBuild` can also be used to build arbitrary dataflow graphs. It lets you connect arbitrary inputs and outputs to each operation.
This way, you can define circuits that read the same boolean multiple times, or allocate qubits dynamically.

In [12]:
from tket2.circuit.build import CircBuild, QAlloc, H, CX, Measure, QFree


# Start building DFG with one qubit input
builder = CircBuild.with_nqb(1)

# Qubits and booleans are identified by their "Wires" in the graph.
# We can get the wire for the single input qubit.
[q0] = builder.inputs()

# And allocate a new qubit, indexing the `Node` output gets the outgoing "Wire".
q1 = builder.add(QAlloc())[0]

# Each operation returns the new wires it creates.
q0 = builder.add(H(q0))[0]
q0, q1 = builder.add(CX(q0, q1))[0, 1]

# Some operations may have different numbers of inputs and outputs.
[q0, b0] = builder.add(Measure(q0))[0, 1]
[q1, b1] = builder.add(Measure(q1))[0, 1]

# And some may have no outputs at all.
builder.add(QFree(q0))
builder.add(QFree(q1))

# To get the final circuit, we need to set the outputs then call finish().
builder.set_outputs(b0, b1)
circ = builder.finish()

circ

```mermaid
graph LR
    subgraph 0 ["(0) DFG"]
        direction LR
        1["(1) Input"]
        2["(2) Output"]
        3["(3) quantum.tket2.QAlloc"]
        4["(4) quantum.tket2.H"]
        5["(5) quantum.tket2.CX"]
        6["(6) quantum.tket2.Measure"]
        7["(7) quantum.tket2.Measure"]
        8["(8) quantum.tket2.QFree"]
        9["(9) quantum.tket2.QFree"]
        1--"0:0<br>qubit"-->4
        3--"0:1<br>qubit"-->5
        4--"0:0<br>qubit"-->5
        5--"0:0<br>qubit"-->6
        5--"1:0<br>qubit"-->7
        6--"0:0<br>qubit"-->8
        6--"1:0<br>[]+[]"-->2
        7--"0:0<br>qubit"-->9
        7--"1:1<br>[]+[]"-->2
    end

```

### Using pytket

We can convert from and to `pytket` circuits.

In [13]:
%pip install pytket

from pytket.circuit import Circuit as PytketCircuit
from pytket.circuit.display import render_circuit_jupyter

Looking in indexes: https://pypi.org/simple, https://cqcpythonrepository.azurewebsites.net/simple/

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
tk1_circ = PytketCircuit(2).H(0).CX(0, 1)
render_circuit_jupyter(tk1_circ)

circ = Tk2Circuit(tk1_circ)
circ

```mermaid
graph LR
    subgraph 0 ["(0) FuncDefn"]
        direction LR
        1["(1) Input"]
        2["(2) Output"]
        3["(3) quantum.tket2.H"]
        4["(4) quantum.tket2.CX"]
        1--"0:0<br>qubit"-->3
        1--"1:1<br>qubit"-->4
        3--"0:0<br>qubit"-->4
        4--"0:0<br>qubit"-->2
        4--"1:1<br>qubit"-->2
    end

```

### Using guppy

Finally, if you have a circuit defined in `guppy` it can be imported directly into a `Tk2Circuit` object.

In [15]:
%pip install guppylang

from guppylang import guppy
from guppylang.module import GuppyModule
from guppylang.prelude import quantum
from guppylang.prelude.builtins import py
from guppylang.prelude.quantum import measure, phased_x, qubit, rz, zz_max

# We define a utility function to convert a GuppyModule to a Tk2Circuit.
# This will be included with guppy in the future.
from utils import guppy_to_circuit # type: ignore

Looking in indexes: https://pypi.org/simple, https://cqcpythonrepository.azurewebsites.net/simple/

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Define a guppy module with a quantum function

module = GuppyModule("test")
module.load(quantum)

@guppy(module)
def my_func(q0: qubit, q1: qubit) -> bool:
    q0 = phased_x(q0, py(math.pi / 2), py(-math.pi / 2))
    q0 = rz(q0, py(math.pi))
    q1 = phased_x(q1, py(math.pi / 2), py(-math.pi / 2))
    q1 = rz(q1, py(math.pi))
    q0, q1 = zz_max(q0, q1)
    _ = measure(q0)
    return measure(q1)

circ = guppy_to_circuit(my_func)
circ

```mermaid
graph LR
    subgraph 1 ["(1) DFG"]
        direction LR
        2["(2) Input"]
        27["(27) Output"]
        4["(4) const:custom:f64(1.5707963267948966)"]
        5["(5) LoadConstant"]
        6["(6) const:custom:f64(-1.5707963267948966)"]
        7["(7) LoadConstant"]
        8["(8) quantum.tket2.PhasedX"]
        9["(9) const:custom:f64(3.141592653589793)"]
        10["(10) LoadConstant"]
        11["(11) quantum.tket2.RzF64"]
        12["(12) const:custom:f64(1.5707963267948966)"]
        13["(13) LoadConstant"]
        14["(14) const:custom:f64(-1.5707963267948966)"]
        15["(15) LoadConstant"]
        16["(16) quantum.tket2.PhasedX"]
        17["(17) const:custom:f64(3.141592653589793)"]
        18["(18) LoadConstant"]
        19["(19) quantum.tket2.RzF64"]
        20["(20) quantum.tket2.ZZMax"]
        23["(23) quantum.tket2.Measure"]
        24["(24) quantum.tket2.QFree"]
        25["(25) quantum.tket2.Measure"]
        26["(26) quantum.tket2.QFree"]
        2--"0:0<br>qubit"-->8
        2--"1:0<br>qubit"-->16
        4--"0:0<br>float64"-->5
        5--"0:1<br>float64"-->8
        6--"0:0<br>float64"-->7
        7--"0:2<br>float64"-->8
        8--"0:0<br>qubit"-->11
        9--"0:0<br>float64"-->10
        10--"0:1<br>float64"-->11
        11--"0:0<br>qubit"-->20
        12--"0:0<br>float64"-->13
        13--"0:1<br>float64"-->16
        14--"0:0<br>float64"-->15
        15--"0:2<br>float64"-->16
        16--"0:0<br>qubit"-->19
        17--"0:0<br>float64"-->18
        18--"0:1<br>float64"-->19
        19--"0:1<br>qubit"-->20
        20--"0:0<br>qubit"-->23
        20--"1:0<br>qubit"-->25
        23--"0:0<br>qubit"-->24
        25--"0:0<br>qubit"-->26
        25--"1:0<br>[]+[]"-->27
    end

```

This can be combined with the pytket conversion to obtain a `pytket` circuit from a guppy definition!

In [21]:
tk1_circ = circ.to_tket1()

render_circuit_jupyter(tk1_circ)